In [347]:
import dask.dataframe as dd
import time
from datetime import datetime
import pandas as pd
import numpy as np
from scipy.stats import mstats
#from progressbar import *
import matplotlib.pyplot as plt
import os
import gc

In [6]:
if not os.path.exists('./Stats_Detail_New/'):
    os.makedirs('./Stats_Detail_New/')
if not os.path.exists('./Weight/'):
    os.makedirs('./Weight/')
if not os.path.exists('./Graph_new/'):
    os.makedirs('./Graph_new/')
if not os.path.exists('./stats/'):
    os.makedirs('./stats/')

In [9]:
Factor_args = {}
#tri -> 排序權重法
Factor_args['weight_method'] = ['tri']
tir_list = [x for x in Factor_args['weight_method'] if 'tri' in x]
Factor_args['weight_method'] = Factor_args['weight_method'] + [x + '_value' for x in tir_list]

In [15]:
Factor_args

{'weight_method': ['tri', 'tri_value'],
 'upper_lower_perc_eq': {'upper': 0.1, 'lower': 0.9},
 'upper_lower_perc_tri': {'upper': 0.5, 'lower': 0.5},
 'PE': {'factors': [], 'longbig': [], 'periods': []}}

In [13]:
Main_Factor_Name = 'PE'
Main_Factor_Long = 0
periods = 1

In [14]:
#eq:平均權重法 -> long/short因子最大的10%，short/long因子最小的10%(90%)
#tri:排序權重法 -> 以中位數切割，排序並給予權重
Factor_args['upper_lower_perc_eq'] =  { "upper" : 0.1,"lower" : 0.9 }
Factor_args['upper_lower_perc_tri'] =  { "upper" : 0.5,"lower" : 0.5 }
Factor_args[Main_Factor_Name] = { "factors" : [],
                                  "longbig" : [],
                                  "periods" : [] }

In [18]:
Factor_args[Main_Factor_Name]['factors'] = [Main_Factor_Name]
Factor_args[Main_Factor_Name]['longbig'] = [Main_Factor_Long] * len(Factor_args[Main_Factor_Name]['factors']) #1是多大的
Factor_args[Main_Factor_Name]['periods'] = [periods]

In [19]:
Factor_args

{'weight_method': ['tri', 'tri_value'],
 'upper_lower_perc_eq': {'upper': 0.1, 'lower': 0.9},
 'upper_lower_perc_tri': {'upper': 0.5, 'lower': 0.5},
 'PE': {'factors': ['PE'], 'longbig': [0], 'periods': [1]}}

# data

In [21]:
#using chunksize to load data (因為資料過大，利用這個分批載入)
reader = pd.read_csv('./Data/output/df_merge_1_new.txt', chunksize=10 ** 6)
reader

In [22]:
#ignore_index = True 以忽略合併時舊的index欄位，改採用自動產生的index
df_merge_final = pd.concat([x for x in reader], ignore_index = True)
df_merge_final

,Code,Date,Close,Volumn,Volumn_a_mean,CAP,Year,Ind,大盤指數,Rm
0,1101,20070102,29.75,4092.0,NaN,934.6,2007,1.0,9055.41,0.012408
1,1101,20070103,29.45,4839.0,262595240.0,925.2,2007,1.0,9051.41,-0.000442
2,1101,20070104,29.35,2318.0,262098608.0,922.1,2007,1.0,9071.09,0.002174
3,1101,20070105,29.05,7197.0,261089444.0,912.6,2007,1.0,8957.97,-0.012470
4,1101,20070108,29.00,6579.0,260653208.0,911.1,2007,1.0,8844.95,-0.012617
...,...,...,...,...,...,...,...,...,...,...
6727271,9962,20190923,10.60,62.0,831716.0,9.6,2019,10.0,20319.98,-0.000975
6727272,9962,20190924,10.60,24.0,831932.0,9.6,2019,10.0,20318.25,-0.000085
6727273,9962,20190925,10.55,35.0,830044.0,9.5,2019,10.0,20235.92,-0.004052
6727274,9962,20190926,10.50,29.0,830756.0,9.5,2019,10.0,20233.00,-0.000144


In [23]:
df_merge_final.count()

Code             6727276
Date             6727276
Close            4593760
Volumn           4593527
Volumn_a_mean    4242522
CAP              4590628
Year             6727276
Ind              6727276
大盤指數             6727276
Rm               6727276
dtype: int64

In [24]:
factor = Factor_args[Main_Factor_Name]['factors'][0]
periods = Factor_args[Main_Factor_Name]['periods'][0]

In [27]:
reader = pd.read_csv('./Data/output/df_merge_2.txt', chunksize = 10 ** 6)
df_factor = pd.concat([x for x in reader], ignore_index = True)

In [29]:
df_factor[['Date', 'Code', factor]]

,Date,Code,PE
0,20070102,1101,13.5
1,20070103,1101,13.4
2,20070104,1101,13.3
3,20070105,1101,13.2
4,20070108,1101,13.2
...,...,...,...
6730419,20190923,9962,NaN
6730420,20190924,9962,NaN
6730421,20190925,9962,NaN
6730422,20190926,9962,NaN


In [32]:
#how = left -> 以left的index為準，只 merge兩者on(Date、Code)的交集
df_merge_final = pd.merge(left = df_merge_final, right = df_factor[['Date', 'Code', factor]], 
                         on = ['Date', 'Code'], how = 'left')
df_merge_final

,Code,Date,Close,Volumn,Volumn_a_mean,CAP,Year,Ind,大盤指數,Rm,PE
0,1101,20070102,29.75,4092.0,NaN,934.6,2007,1.0,9055.41,0.012408,13.5
1,1101,20070103,29.45,4839.0,262595240.0,925.2,2007,1.0,9051.41,-0.000442,13.4
2,1101,20070104,29.35,2318.0,262098608.0,922.1,2007,1.0,9071.09,0.002174,13.3
3,1101,20070105,29.05,7197.0,261089444.0,912.6,2007,1.0,8957.97,-0.012470,13.2
4,1101,20070108,29.00,6579.0,260653208.0,911.1,2007,1.0,8844.95,-0.012617,13.2
...,...,...,...,...,...,...,...,...,...,...,...
6727271,9962,20190923,10.60,62.0,831716.0,9.6,2019,10.0,20319.98,-0.000975,NaN
6727272,9962,20190924,10.60,24.0,831932.0,9.6,2019,10.0,20318.25,-0.000085,NaN
6727273,9962,20190925,10.55,35.0,830044.0,9.5,2019,10.0,20235.92,-0.004052,NaN
6727274,9962,20190926,10.50,29.0,830756.0,9.5,2019,10.0,20233.00,-0.000144,NaN


In [34]:
df_merge_final.count()

Code             6727276
Date             6727276
Close            4593760
Volumn           4593527
Volumn_a_mean    4242522
CAP              4590628
Year             6727276
Ind              6727276
大盤指數             6727276
Rm               6727276
PE               3801131
dtype: int64

In [35]:
del df_factor

In [36]:
#shift -> 向上或向下平移，缺少的值以NaN填補
df_merge_final.groupby('Code')['CAP'].shift(1)

0            NaN
1          934.6
2          925.2
3          922.1
4          912.6
           ...  
6727271      9.7
6727272      9.6
6727273      9.6
6727274      9.5
6727275      9.5
Name: CAP, Length: 6727276, dtype: float64

In [37]:
df_merge_final['CAP_t-1'] = df_merge_final.groupby('Code')['CAP'].shift(1)

,Code,Date,Close,Volumn,Volumn_a_mean,CAP,Year,Ind,大盤指數,Rm,PE,CAP_t-1
0,1101,20070102,29.75,4092.0,NaN,934.6,2007,1.0,9055.41,0.012408,13.5,NaN
1,1101,20070103,29.45,4839.0,262595240.0,925.2,2007,1.0,9051.41,-0.000442,13.4,934.6
2,1101,20070104,29.35,2318.0,262098608.0,922.1,2007,1.0,9071.09,0.002174,13.3,925.2
3,1101,20070105,29.05,7197.0,261089444.0,912.6,2007,1.0,8957.97,-0.012470,13.2,922.1
4,1101,20070108,29.00,6579.0,260653208.0,911.1,2007,1.0,8844.95,-0.012617,13.2,912.6
...,...,...,...,...,...,...,...,...,...,...,...,...
6727271,9962,20190923,10.60,62.0,831716.0,9.6,2019,10.0,20319.98,-0.000975,NaN,9.7
6727272,9962,20190924,10.60,24.0,831932.0,9.6,2019,10.0,20318.25,-0.000085,NaN,9.6
6727273,9962,20190925,10.55,35.0,830044.0,9.5,2019,10.0,20235.92,-0.004052,NaN,9.6
6727274,9962,20190926,10.50,29.0,830756.0,9.5,2019,10.0,20233.00,-0.000144,NaN,9.5


In [40]:
#main factor -> PE -> 計算T-1、T-2
factor_list = Factor_args[Main_Factor_Name]['factors']
for factor in factor_list :
    df_merge_final[factor+'_t-1']= df_merge_final.groupby('Code')[factor].shift(1)
    df_merge_final[factor+'_t-2']= df_merge_final.groupby('Code')[factor].shift(2)

In [42]:
# shift 收盤價方便計算投組損益 (why?)
df_merge_final['Close_t+1'] = df_merge_final.groupby('Code')['Close'].shift(-1)
df_merge_final['Close_t-1'] = df_merge_final.groupby('Code')['Close'].shift(1)
df_merge_final

,Code,Date,Close,Volumn,Volumn_a_mean,CAP,Year,Ind,大盤指數,Rm,PE,CAP_t-1,PE_t-1,PE_t-2,Close_t+1,Close_t-1
0,1101,20070102,29.75,4092.0,NaN,934.6,2007,1.0,9055.41,0.012408,13.5,NaN,NaN,NaN,29.45,NaN
1,1101,20070103,29.45,4839.0,262595240.0,925.2,2007,1.0,9051.41,-0.000442,13.4,934.6,13.5,NaN,29.35,29.75
2,1101,20070104,29.35,2318.0,262098608.0,922.1,2007,1.0,9071.09,0.002174,13.3,925.2,13.4,13.5,29.05,29.45
3,1101,20070105,29.05,7197.0,261089444.0,912.6,2007,1.0,8957.97,-0.012470,13.2,922.1,13.3,13.4,29.00,29.35
4,1101,20070108,29.00,6579.0,260653208.0,911.1,2007,1.0,8844.95,-0.012617,13.2,912.6,13.2,13.3,30.25,29.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6727271,9962,20190923,10.60,62.0,831716.0,9.6,2019,10.0,20319.98,-0.000975,NaN,9.7,NaN,NaN,10.60,10.70
6727272,9962,20190924,10.60,24.0,831932.0,9.6,2019,10.0,20318.25,-0.000085,NaN,9.6,NaN,NaN,10.55,10.60
6727273,9962,20190925,10.55,35.0,830044.0,9.5,2019,10.0,20235.92,-0.004052,NaN,9.6,NaN,NaN,10.50,10.60
6727274,9962,20190926,10.50,29.0,830756.0,9.5,2019,10.0,20233.00,-0.000144,NaN,9.5,NaN,NaN,10.40,10.55


In [45]:
np.array(df_merge_final['Date'].unique())[2]

20070104

In [46]:
# 調整開始時間
date_begin = np.array(df_merge_final['Date'].unique())[2]
df_merge_final = df_merge_final[df_merge_final['Date'] >= date_begin]
df_merge_final

,Code,Date,Close,Volumn,Volumn_a_mean,CAP,Year,Ind,大盤指數,Rm,PE,CAP_t-1,PE_t-1,PE_t-2,Close_t+1,Close_t-1
2,1101,20070104,29.35,2318.0,262098608.0,922.1,2007,1.0,9071.09,0.002174,13.3,925.2,13.4,13.5,29.05,29.45
3,1101,20070105,29.05,7197.0,261089444.0,912.6,2007,1.0,8957.97,-0.012470,13.2,922.1,13.3,13.4,29.00,29.35
4,1101,20070108,29.00,6579.0,260653208.0,911.1,2007,1.0,8844.95,-0.012617,13.2,912.6,13.2,13.3,30.25,29.05
5,1101,20070109,30.25,10074.0,260225256.0,950.3,2007,1.0,8905.89,0.006890,13.7,911.1,13.2,13.2,29.75,29.00
6,1101,20070110,29.75,13399.0,259865000.0,934.6,2007,1.0,8801.28,-0.011746,13.5,950.3,13.7,13.2,29.05,30.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6727271,9962,20190923,10.60,62.0,831716.0,9.6,2019,10.0,20319.98,-0.000975,NaN,9.7,NaN,NaN,10.60,10.70
6727272,9962,20190924,10.60,24.0,831932.0,9.6,2019,10.0,20318.25,-0.000085,NaN,9.6,NaN,NaN,10.55,10.60
6727273,9962,20190925,10.55,35.0,830044.0,9.5,2019,10.0,20235.92,-0.004052,NaN,9.6,NaN,NaN,10.50,10.60
6727274,9962,20190926,10.50,29.0,830756.0,9.5,2019,10.0,20233.00,-0.000144,NaN,9.5,NaN,NaN,10.40,10.55


In [48]:
#以5000萬平均volumn為一篩選標準
df_merge_final['Volumn_a_select'] = (df_merge_final['Volumn_a_mean'] >=50000000)*1
df_merge_final

C:\Users\user10\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Code,Date,Close,Volumn,Volumn_a_mean,CAP,Year,Ind,大盤指數,Rm,PE,CAP_t-1,PE_t-1,PE_t-2,Close_t+1,Close_t-1,Volumn_a_select
2,1101,20070104,29.35,2318.0,262098608.0,922.1,2007,1.0,9071.09,0.002174,13.3,925.2,13.4,13.5,29.05,29.45,1
3,1101,20070105,29.05,7197.0,261089444.0,912.6,2007,1.0,8957.97,-0.012470,13.2,922.1,13.3,13.4,29.00,29.35,1
4,1101,20070108,29.00,6579.0,260653208.0,911.1,2007,1.0,8844.95,-0.012617,13.2,912.6,13.2,13.3,30.25,29.05,1
5,1101,20070109,30.25,10074.0,260225256.0,950.3,2007,1.0,8905.89,0.006890,13.7,911.1,13.2,13.2,29.75,29.00,1
6,1101,20070110,29.75,13399.0,259865000.0,934.6,2007,1.0,8801.28,-0.011746,13.5,950.3,13.7,13.2,29.05,30.25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6727271,9962,20190923,10.60,62.0,831716.0,9.6,2019,10.0,20319.98,-0.000975,NaN,9.7,NaN,NaN,10.60,10.70,0
6727272,9962,20190924,10.60,24.0,831932.0,9.6,2019,10.0,20318.25,-0.000085,NaN,9.6,NaN,NaN,10.55,10.60,0
6727273,9962,20190925,10.55,35.0,830044.0,9.5,2019,10.0,20235.92,-0.004052,NaN,9.6,NaN,NaN,10.50,10.60,0
6727274,9962,20190926,10.50,29.0,830756.0,9.5,2019,10.0,20233.00,-0.000144,NaN,9.5,NaN,NaN,10.40,10.55,0


In [49]:
# 剔除Close , factor(PE) t-1 t-2 為na的資料   
df_merge_final = df_merge_final[pd.isna(df_merge_final['Close']) == False]
df_merge_final = df_merge_final[pd.isna(df_merge_final['CAP_t-1']) == False]
df_merge_final = df_merge_final[pd.isna(df_merge_final[factor+'_t-1']) == False]
df_merge_final

,Code,Date,Close,Volumn,Volumn_a_mean,CAP,Year,Ind,大盤指數,Rm,PE,CAP_t-1,PE_t-1,PE_t-2,Close_t+1,Close_t-1,Volumn_a_select
2,1101,20070104,29.35,2318.0,262098608.0,922.1,2007,1.0,9071.09,0.002174,13.3,925.2,13.4,13.5,29.05,29.45,1
3,1101,20070105,29.05,7197.0,261089444.0,912.6,2007,1.0,8957.97,-0.012470,13.2,922.1,13.3,13.4,29.00,29.35,1
4,1101,20070108,29.00,6579.0,260653208.0,911.1,2007,1.0,8844.95,-0.012617,13.2,912.6,13.2,13.3,30.25,29.05,1
5,1101,20070109,30.25,10074.0,260225256.0,950.3,2007,1.0,8905.89,0.006890,13.7,911.1,13.2,13.2,29.75,29.00,1
6,1101,20070110,29.75,13399.0,259865000.0,934.6,2007,1.0,8801.28,-0.011746,13.5,950.3,13.7,13.2,29.05,30.25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6727174,9962,20190506,12.05,85.0,2497884.0,10.9,2019,10.0,19441.43,-0.017950,12.1,10.9,12.1,12.1,12.00,12.10,0
6727175,9962,20190507,12.00,32.0,2495608.0,10.8,2019,10.0,19602.03,0.008261,12.0,10.9,12.1,12.1,12.00,12.05,0
6727176,9962,20190508,12.00,153.0,2501720.0,10.8,2019,10.0,19488.95,-0.005769,12.0,10.8,12.0,12.1,11.90,12.00,0
6727177,9962,20190509,11.90,86.0,2503732.0,10.7,2019,10.0,19149.90,-0.017397,11.9,10.8,12.0,12.0,11.80,12.00,0


In [53]:
#setting first day of every year
first_date = np.array(df_merge_final.groupby('Year')['Date'].min())
first_date

array([20070104, 20080102, 20090105, 20100104, 20110103, 20120102,
       20130102, 20140102, 20150105, 20160104, 20170103, 20180102,
       20190102], dtype=int64)

In [58]:
#依照所有first date的資料，篩選出這些股票數據 -> True or False
[x in first_date for x in df_merge_final['Date']]

[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 

In [59]:
df_Year = df_merge_final[[x in first_date for x in df_merge_final['Date']]]
df_Year

,Code,Date,Close,Volumn,Volumn_a_mean,CAP,Year,Ind,大盤指數,Rm,PE,CAP_t-1,PE_t-1,PE_t-2,Close_t+1,Close_t-1,Volumn_a_select
2,1101,20070104,29.35,2318.0,262098608.0,922.1,2007,1.0,9071.09,0.002174,13.3,925.2,13.4,13.5,29.05,29.45,1
247,1101,20080102,45.05,21807.0,676533268.0,1463.5,2008,1.0,9845.42,-0.021540,18.3,1461.9,16.8,17.3,44.80,45.00,1
496,1101,20090105,27.65,20441.0,866203632.0,910.3,2009,1.0,5862.14,0.023327,13.3,887.2,14.4,14.2,27.45,26.95,1
747,1101,20100104,34.30,8299.0,564009224.0,1129.2,2010,1.0,10528.26,0.002412,15.8,1119.3,17.7,17.7,35.50,34.00,1
998,1101,20110103,33.40,17962.0,411637184.0,1233.2,2011,1.0,11998.76,0.005885,13.6,1211.0,16.7,16.5,33.15,32.80,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6725869,9962,20140102,12.10,202.0,1870096.0,10.9,2014,10.0,12724.76,0.000119,11.0,10.9,12.1,12.4,12.10,12.10,0
6726117,9962,20150105,11.60,75.0,2854940.0,10.5,2015,10.0,14121.92,-0.003562,10.6,10.6,10.7,10.7,11.55,11.70,0
6726605,9962,20170103,9.99,1.0,1764120.0,9.0,2017,10.0,15288.17,0.002094,20.0,9.0,16.7,16.6,9.97,9.99,0
6726851,9962,20180102,10.30,39.0,1449360.0,9.3,2018,10.0,18351.03,0.006377,17.2,9.2,34.1,34.2,10.35,10.20,0


In [60]:
df_Year = df_Year.sort_values(['Code', 'Year']).reset_index(drop = True)
df_Year

,Code,Date,Close,Volumn,Volumn_a_mean,CAP,Year,Ind,大盤指數,Rm,PE,CAP_t-1,PE_t-1,PE_t-2,Close_t+1,Close_t-1,Volumn_a_select
0,1101,20070104,29.35,2318.0,262098608.0,922.1,2007,1.0,9071.09,0.002174,13.3,925.2,13.4,13.5,29.05,29.45,1
1,1101,20080102,45.05,21807.0,676533268.0,1463.5,2008,1.0,9845.42,-0.021540,18.3,1461.9,16.8,17.3,44.80,45.00,1
2,1101,20090105,27.65,20441.0,866203632.0,910.3,2009,1.0,5862.14,0.023327,13.3,887.2,14.4,14.2,27.45,26.95,1
3,1101,20100104,34.30,8299.0,564009224.0,1129.2,2010,1.0,10528.26,0.002412,15.8,1119.3,17.7,17.7,35.50,34.00,1
4,1101,20110103,33.40,17962.0,411637184.0,1233.2,2011,1.0,11998.76,0.005885,13.6,1211.0,16.7,16.5,33.15,32.80,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15245,9962,20140102,12.10,202.0,1870096.0,10.9,2014,10.0,12724.76,0.000119,11.0,10.9,12.1,12.4,12.10,12.10,0
15246,9962,20150105,11.60,75.0,2854940.0,10.5,2015,10.0,14121.92,-0.003562,10.6,10.6,10.7,10.7,11.55,11.70,0
15247,9962,20170103,9.99,1.0,1764120.0,9.0,2017,10.0,15288.17,0.002094,20.0,9.0,16.7,16.6,9.97,9.99,0
15248,9962,20180102,10.30,39.0,1449360.0,9.3,2018,10.0,18351.03,0.006377,17.2,9.2,34.1,34.2,10.35,10.20,0


In [69]:
#count how many times when Volumn > 5000萬 every year
temp = df_Year[df_Year['Volumn_a_select'] == 1]
temp.groupby('Year')['Code'].count()

Year
2007    359
2008    515
2009    288
2010    393
2011    469
2012    342
2013    247
2014    312
2015    368
2016    322
2017    275
2018    377
2019    333
Name: Code, dtype: int64

In [70]:
temp

,Code,Date,Close,Volumn,Volumn_a_mean,CAP,Year,Ind,大盤指數,Rm,PE,CAP_t-1,PE_t-1,PE_t-2,Close_t+1,Close_t-1,Volumn_a_select
0,1101,20070104,29.35,2318.0,262098608.0,922.1,2007,1.0,9071.09,0.002174,13.3,925.2,13.4,13.5,29.05,29.45,1
1,1101,20080102,45.05,21807.0,676533268.0,1463.5,2008,1.0,9845.42,-0.021540,18.3,1461.9,16.8,17.3,44.80,45.00,1
2,1101,20090105,27.65,20441.0,866203632.0,910.3,2009,1.0,5862.14,0.023327,13.3,887.2,14.4,14.2,27.45,26.95,1
3,1101,20100104,34.30,8299.0,564009224.0,1129.2,2010,1.0,10528.26,0.002412,15.8,1119.3,17.7,17.7,35.50,34.00,1
4,1101,20110103,33.40,17962.0,411637184.0,1233.2,2011,1.0,11998.76,0.005885,13.6,1211.0,16.7,16.5,33.15,32.80,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15204,9955,20070104,55.60,869.0,62365396.0,41.8,2007,20.0,9071.09,0.002174,12.2,42.6,12.4,12.6,56.10,56.60,1
15205,9955,20080102,71.00,2246.0,142955868.0,58.3,2008,20.0,9845.42,-0.021540,20.3,55.3,20.0,18.7,69.50,67.40,1
15206,9955,20090105,31.00,460.0,56059828.0,26.5,2009,20.0,5862.14,0.023327,10.8,26.0,8.9,9.0,30.50,30.40,1
15207,9955,20100104,63.60,1725.0,72603312.0,66.5,2010,20.0,10528.26,0.002412,18.9,65.1,26.4,26.4,62.80,62.20,1


In [72]:
#降序排名CAP(t-1)
#再選取CAP(t-1)前200小的
temp['MV_t-1_Rank'] = temp.groupby('Year')['CAP_t-1'].rank(ascending = False)
temp['MV_Select'] = (temp['MV_t-1_Rank'] < 201)*1
temp

C:\Users\user10\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user10\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Code,Date,Close,Volumn,Volumn_a_mean,CAP,Year,Ind,大盤指數,Rm,PE,CAP_t-1,PE_t-1,PE_t-2,Close_t+1,Close_t-1,Volumn_a_select,MV_t-1_Rank,MV_Select
0,1101,20070104,29.35,2318.0,262098608.0,922.1,2007,1.0,9071.09,0.002174,13.3,925.2,13.4,13.5,29.05,29.45,1,44.0,1
1,1101,20080102,45.05,21807.0,676533268.0,1463.5,2008,1.0,9845.42,-0.021540,18.3,1461.9,16.8,17.3,44.80,45.00,1,29.0,1
2,1101,20090105,27.65,20441.0,866203632.0,910.3,2009,1.0,5862.14,0.023327,13.3,887.2,14.4,14.2,27.45,26.95,1,28.0,1
3,1101,20100104,34.30,8299.0,564009224.0,1129.2,2010,1.0,10528.26,0.002412,15.8,1119.3,17.7,17.7,35.50,34.00,1,34.0,1
4,1101,20110103,33.40,17962.0,411637184.0,1233.2,2011,1.0,11998.76,0.005885,13.6,1211.0,16.7,16.5,33.15,32.80,1,36.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15204,9955,20070104,55.60,869.0,62365396.0,41.8,2007,20.0,9071.09,0.002174,12.2,42.6,12.4,12.6,56.10,56.60,1,335.0,0
15205,9955,20080102,71.00,2246.0,142955868.0,58.3,2008,20.0,9845.42,-0.021540,20.3,55.3,20.0,18.7,69.50,67.40,1,353.5,0
15206,9955,20090105,31.00,460.0,56059828.0,26.5,2009,20.0,5862.14,0.023327,10.8,26.0,8.9,9.0,30.50,30.40,1,262.0,0
15207,9955,20100104,63.60,1725.0,72603312.0,66.5,2010,20.0,10528.26,0.002412,18.9,65.1,26.4,26.4,62.80,62.20,1,311.0,0


In [76]:
temp = temp[['Code','Year','MV_Select','Volumn_a_select','CAP_t-1','Ind']]
temp_m = temp[temp['MV_Select'] == 1]
temp_m

,Code,Year,MV_Select,Volumn_a_select,CAP_t-1,Ind
0,1101,2007,1,1,925.2,1.0
1,1101,2008,1,1,1461.9,1.0
2,1101,2009,1,1,887.2,1.0
3,1101,2010,1,1,1119.3,1.0
4,1101,2011,1,1,1211.0,1.0
...,...,...,...,...,...,...
15160,9945,2017,1,1,512.8,20.0
15161,9945,2018,1,1,534.2,20.0
15200,9951,2016,1,1,125.1,5.0
15201,9951,2017,1,1,143.8,5.0


In [78]:
temp_m.groupby('Year')['Code'].count()

Year
2007    200
2008    200
2009    200
2010    200
2011    200
2012    200
2013    200
2014    201
2015    200
2016    200
2017    200
2018    200
2019    201
Name: Code, dtype: int64

# 策略:排序權重法(考慮產業)

In [84]:
#數量前10大產業
def select_top10_ind(data):
    data_MV = data[data.MV_Select == 1]
    #倒敘排列大小順序
    select_ind = data_MV.groupby('Ind')['Code'].count().sort_values(ascending = False) 
    # why need to > 4??
    select_ind = select_ind[select_ind > 4]
    # 前10大之Ind -> to list
    select_ind = select_ind.index[0:10].tolist()
    #return 1 or 0
    return [ ((x in select_ind) & ( y in data_MV['Code'].tolist())) * 1 
            for x, y in zip(data['Ind'], data['Code']) ]

In [85]:
temp_m = temp_m.sort_values(['Year','Ind','Code']).reset_index(drop=True)

In [98]:
a = temp_m.groupby('Ind')['Code'].count().sort_values(ascending = False) 
a[a>4]

Ind
24.0    368
25.0    271
17.0    265
26.0    213
28.0    194
27.0    129
20.0    126
14.0     99
15.0     93
3.0      89
31.0     81
5.0      72
21.0     65
22.0     64
4.0      58
10.0     54
11.0     54
2.0      52
18.0     44
12.0     41
29.0     38
1.0      30
30.0     26
16.0     22
23.0     14
6.0      13
9.0      11
8.0       8
32.0      7
Name: Code, dtype: int64

In [99]:
ab = a[a>4].index[0:10].tolist()
ab

[24.0, 25.0, 17.0, 26.0, 28.0, 27.0, 20.0, 14.0, 15.0, 3.0]

In [104]:
len([((x in ab) & ( y in temp_m['Code'].tolist())) * 1 
            for x, y in zip(temp_m['Ind'], temp_m['Code'])])

2602

In [105]:
#標注每年200檔市值最大的個股，數量前10大產業
Ind_select = temp_m.groupby('Year').apply(lambda x : select_top10_ind(x) ).tolist()
Ind_select = [ x for y in Ind_select for x in y ]
temp_m['Ind_Select'] = Ind_select
temp_m

,Code,Year,MV_Select,Volumn_a_select,CAP_t-1,Ind,Ind_Select
0,1101,2007,1,1,925.2,1.0,0
1,1102,2007,1,1,807.7,1.0,0
2,1103,2007,1,1,143.7,1.0,0
3,1201,2007,1,1,116.4,2.0,0
4,1216,2007,1,1,1073.3,2.0,0
...,...,...,...,...,...,...,...
2597,2354,2019,1,1,855.8,31.0,0
2598,2360,2019,1,1,491.7,31.0,0
2599,2474,2019,1,1,1733.4,31.0,0
2600,3665,2019,1,1,265.4,31.0,0


In [108]:
#1916家為前10大產業
sum(temp_m['Ind_Select'])

1916

In [109]:
#各年度前10產業數量
temp_i = temp_m [temp_m.Ind_Select ==1]
temp_i.groupby('Year')['Code'].count()

Year
2007    155
2008    149
2009    143
2010    151
2011    147
2012    137
2013    144
2014    142
2015    145
2016    149
2017    155
2018    151
2019    148
Name: Code, dtype: int64

In [110]:
temp_i

,Code,Year,MV_Select,Volumn_a_select,CAP_t-1,Ind,Ind_Select
20,1532,2007,1,1,145.0,10.0,1
21,2002,2007,1,1,3816.5,10.0,1
22,2006,2007,1,1,267.4,10.0,1
23,2014,2007,1,1,239.6,10.0,1
24,2015,2007,1,1,228.4,10.0,1
...,...,...,...,...,...,...,...
2589,4943,2019,1,1,116.5,28.0,1
2590,4958,2019,1,1,645.4,28.0,1
2591,6213,2019,1,1,151.5,28.0,1
2592,6269,2019,1,1,246.4,28.0,1


In [111]:
#計算10產業市值權重
#依照year、ind分類，去作加總
temp_i.groupby(['Year', 'Ind'])['CAP_t-1'].sum()

Year  Ind 
2007  10.0     5169.2
      14.0     2367.0
      15.0     3548.2
      17.0    29840.9
      24.0    36331.3
               ...   
2019  24.0    76102.3
      25.0    12324.6
      26.0     9014.0
      27.0    17097.7
      28.0     9878.1
Name: CAP_t-1, Length: 130, dtype: float64

In [112]:
#依照年去作加總
temp_i.groupby(['Year'])['CAP_t-1'].sum()

Year
2007    141323.2
2008    125830.0
2009     65955.7
2010    135633.6
2011    142855.0
2012    106401.6
2013    128851.8
2014    149759.0
2015    159597.6
2016    160704.7
2017    176402.9
2018    212933.0
2019    195977.7
Name: CAP_t-1, dtype: float64

In [114]:
#相除 -> 權重
Ind_Cap_Weight = temp_i.groupby(['Year','Ind'])['CAP_t-1'].sum()/temp_i.groupby(['Year'])['CAP_t-1'].sum()
Ind_Cap_Weight

Year  Ind 
2007  10.0    0.036577
      14.0    0.016749
      15.0    0.025107
      17.0    0.211154
      24.0    0.257080
                ...   
2019  24.0    0.388321
      25.0    0.062888
      26.0    0.045995
      27.0    0.087243
      28.0    0.050404
Name: CAP_t-1, Length: 130, dtype: float64

In [115]:
Ind_Cap_Weight[2007]

Ind
10.0    0.036577
14.0    0.016749
15.0    0.025107
17.0    0.211154
24.0    0.257080
25.0    0.114157
26.0    0.075464
27.0    0.101173
28.0    0.047418
31.0    0.115121
Name: CAP_t-1, dtype: float64

In [116]:
Ind_Cap_Weight[2007][10]

0.03657715081458674

In [117]:
#新增一欄權重 (if ind select == 1 -> 10大產業)
temp_m["Ind_CAP_Weight"] = [Ind_Cap_Weight[x][y] if z == 1 else 0 for x, y, z in 
                                 zip(temp_m["Year"], temp_m["Ind"], temp_m["Ind_Select"])]
temp_m

,Code,Year,MV_Select,Volumn_a_select,CAP_t-1,Ind,Ind_Select,Ind_CAP_Weight
0,1101,2007,1,1,925.2,1.0,0,0.0
1,1102,2007,1,1,807.7,1.0,0,0.0
2,1103,2007,1,1,143.7,1.0,0,0.0
3,1201,2007,1,1,116.4,2.0,0,0.0
4,1216,2007,1,1,1073.3,2.0,0,0.0
...,...,...,...,...,...,...,...,...
2597,2354,2019,1,1,855.8,31.0,0,0.0
2598,2360,2019,1,1,491.7,31.0,0,0.0
2599,2474,2019,1,1,1733.4,31.0,0,0.0
2600,3665,2019,1,1,265.4,31.0,0,0.0


In [119]:
df_merge_Year = pd.merge(left = df_merge_final, 
                         right  =temp_m[['Year', 'Code', 'MV_Select', 'Ind_Select', 'Ind_CAP_Weight']], 
                         on=['Year', 'Code'],how = 'left')
df_merge_Year

,Code,Date,Close,Volumn,Volumn_a_mean,CAP,Year,Ind,大盤指數,Rm,PE,CAP_t-1,PE_t-1,PE_t-2,Close_t+1,Close_t-1,Volumn_a_select,MV_Select,Ind_Select,Ind_CAP_Weight
0,1101,20070104,29.35,2318.0,262098608.0,922.1,2007,1.0,9071.09,0.002174,13.3,925.2,13.4,13.5,29.05,29.45,1,1.0,0.0,0.0
1,1101,20070105,29.05,7197.0,261089444.0,912.6,2007,1.0,8957.97,-0.012470,13.2,922.1,13.3,13.4,29.00,29.35,1,1.0,0.0,0.0
2,1101,20070108,29.00,6579.0,260653208.0,911.1,2007,1.0,8844.95,-0.012617,13.2,912.6,13.2,13.3,30.25,29.05,1,1.0,0.0,0.0
3,1101,20070109,30.25,10074.0,260225256.0,950.3,2007,1.0,8905.89,0.006890,13.7,911.1,13.2,13.2,29.75,29.00,1,1.0,0.0,0.0
4,1101,20070110,29.75,13399.0,259865000.0,934.6,2007,1.0,8801.28,-0.011746,13.5,950.3,13.7,13.2,29.05,30.25,1,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3797850,9962,20190506,12.05,85.0,2497884.0,10.9,2019,10.0,19441.43,-0.017950,12.1,10.9,12.1,12.1,12.00,12.10,0,NaN,NaN,NaN
3797851,9962,20190507,12.00,32.0,2495608.0,10.8,2019,10.0,19602.03,0.008261,12.0,10.9,12.1,12.1,12.00,12.05,0,NaN,NaN,NaN
3797852,9962,20190508,12.00,153.0,2501720.0,10.8,2019,10.0,19488.95,-0.005769,12.0,10.8,12.0,12.1,11.90,12.00,0,NaN,NaN,NaN
3797853,9962,20190509,11.90,86.0,2503732.0,10.7,2019,10.0,19149.90,-0.017397,11.9,10.8,12.0,12.0,11.80,12.00,0,NaN,NaN,NaN


In [120]:
del df_merge_final, temp_m
gc.collect()

83

# 指標設定function

In [274]:
#持有總值(Amount)、numbers of stocks(除以1000)
def Cal_Amount(df,factor,weight_method):
    df[factor+'_Nstock_'+weight_method] = [initial_account/2*x/y/1000 for x,y in zip(df[factor+'_Weight_'+weight_method],
                                                                                     df['Close'])]
    df[factor+'_Nstock_'+weight_method] = df[factor+'_Nstock_'+weight_method].apply(np.ceil)
    df[factor+'_Amount_'+weight_method] = df[factor+'_Nstock_'+weight_method]*df['Close']*1000    
    return df[factor+'_Nstock_'+weight_method],df[factor+'_Amount_'+weight_method]
#Profit
def Cal_Profit(df, factor, weight_method):
    df[factor+'_Profit_'+weight_method] = ((df['Close_t+1']-df['Close']) * df[factor+'_Nstock_'+weight_method])*1000
    if 'short_index' in weight_method:
        #Rm -> 台灣發行量加權股價報酬指數 pct change
        Rm = df['Rm'].mode()
        Profit = df[factor + '_Profit_' + weight_method].sum() - initial_account * Rm/2
    elif 'long index' in weight_method:
        Rm = df['Rm'].mode()
        Profit = df[factor + '_Profit_' + weight_method].sum() + initial_account * Rm/2
    else:
        Profit = df[factor+'_Profit_'+weight_method].sum()
    return Profit

#Tax Cost
def Cal_Cost(df, factor, weight_method):
    #今天手上的股票數量-昨天的，再*股價
    holdsell_shortmore = df['Close']*(df[factor+'_Nstock_'+weight_method] - df[factor+'_Nstock_'+weight_method+'_t-1'])
    #今天的量比昨天少(<0) -> 賣出股票 -> tax
    holdsell_shortmore = abs(holdsell_shortmore[holdsell_shortmore < 0])
    df[factor + '_Tax_' + weight_method] = holdsell_shortmore
    trade_tax = np.sum(holdsell_shortmore*0.003*1000)
    return trade_tax

#Turnover(周轉率)
def Cal_Turnover(df,factor,weight_method):
    Turnover = np.sum(abs(df['Close']*(df[factor+'_Nstock_'+weight_method] - df[factor+'_Nstock_'+weight_method+'_t-1'])))*1000
    return Turnover

#some setting to calculate 因子值加權法
#極端值調整平均數 -> 去極值
#if s is all nan -> return s
#else -> 調整後1%、前1%的數值
def winsorize(s):
    if  sum(pd.isna(s) == True) == len(s):
        return s
    else:   
        return mstats.winsorize(s, limits=[0.01, 0.01])

#normalize to 0~1 range
def min_max(s):
    if sum(pd.isna(s) == True) == len(s):
        return s
    else:   
        return (s - np.min(s)) / (np.max(s) - np.min(s))

#策略:因子值加權、因子值排序加權
def cal_3group_weight_tri(df, factor, longbid, short_index = False, long_index = False, value = False):
    length = len(df)
    #因子值加權
    if value == True:
        #極端值調整平均數 -> 去極值
        df[factor+'_t-1'] = winsorize(df[factor+'_t-1'])
        #標準化 -> 0~1 range
        df[factor+'_t-1'] = min_max(df[factor+'_t-1'])
        #從中位數切一半 -> upper、lower
        median = df[factor+'_t-1'].median()
        #upper的值
        upper = df[df[factor + '_t-1'] >= median][factor + '_t-1']
        #lower的值
        lower = df[df[factor + '_t-1'] < median][factor + '_t-1']
        large_Rank = upper
        # why 1 -lower???
        small_Rank = 1 - lower
        print(small_Rank)
        print(large_Rank)
    #因子值排序加權
    else:
        length = len(df)
        #從中間切一半
        #middle = 100
        middle = np.ceil(length /2)
        #upper、lower
        #rank -> 200大排序 -> 切一半 100/100
        upper = df[df[factor + '_Rank'] <= middle][factor + '_Rank'] 
        lower = df[df[factor + '_Rank'] > middle][factor + '_Rank']
        #數值最大的逆著排 -> 100 99 98....1   #最大是一
        large_Rank = middle + 1 - upper 
        #因子數值最小的順著排 -> 1 2 3....100 #最小是一
        small_Rank = lower - middle
    #買因子大的，賣因子小的 (一買一賣 -> delta對消的概念)
    if longbid == 1:
        #買大因子，且放空指數 -> 因子小的權重為0
        if short_index == True:
            large_Rank = large_Rank/large_Rank.sum()
            small_Rank = small_Rank/small_Rank.sum() * 0
        #賣小因子，且做多指數 -> 大因子權重為0
        elif long_index == True:
            large_Rank = large_Rank/large_Rank.sum() * 0
            small_Rank = small_Rank/small_Rank.sum()
        else:
            large_Rank = large_Rank/large_Rank.sum()
            small_Rank = small_Rank/small_Rank.sum()
        return pd.concat([large_Rank, -small_Rank])
    #賣因子大的，買因子小的 (一賣一買 -> delta對消的概念)
    else:
        #買小因子，且放空指數 -> 因子大的權重為0
        if short_index == True: 
            large_Rank = large_Rank/large_Rank.sum() * 0 
            small_Rank = small_Rank/small_Rank.sum() 
        #賣大因子，且做多指數 -> 因子小的權重為0
        elif long_index == True:
            large_Rank = large_Rank/large_Rank.sum() 
            small_Rank = small_Rank/small_Rank.sum() * 0 
        else:
            large_Rank = large_Rank/large_Rank.sum()  
            small_Rank = small_Rank/small_Rank.sum()  
        return pd.concat([-large_Rank, small_Rank])

#out of group
#qcut -> 依照X分位數分群資料
#找出今天與昨天不一樣的(差異大的)
def out_of_group(data, n):
    gt1 = pd.qcut(data[factor + '_Rank_t-1'], n, labels = False)
    gt2 = pd.qcut(data[factor + '_Rank'], n, labels = False)
    return (gt1 != gt2) * 1 


In [226]:
g1 = pd.qcut(df_merge_Year['CAP_t-1'], 4, labels = False)

In [229]:
g2 = pd.qcut(df_merge_Year['CAP'], 4, labels = False)
np.sum(g1 != g2)

46377

In [230]:
g1 != g2

0          False
1          False
2          False
3          False
4          False
           ...  
3797850    False
3797851    False
3797852    False
3797853    False
3797854    False
Length: 3797855, dtype: bool

In [204]:
df = df_merge_Year['PE'][0:3797854]

In [207]:
length = len(df)
        #從中間切一半
middle = np.ceil(length /2)
        #upper、lower
upper = df[df <= middle]
lower = df[df > middle]
        #數值最大的逆著排 -> 100 99 98 #最大是一
large_Rank = middle + 1 - upper 
        #因子數值最小的順著排 -> 1 2 3 #最小是一
small_Rank = lower - middle

In [210]:
large_Rank

0          1898914.7
1          1898914.8
2          1898914.8
3          1898914.3
4          1898914.5
             ...    
3797849    1898915.9
3797850    1898915.9
3797851    1898916.0
3797852    1898916.0
3797853    1898916.1
Name: PE, Length: 3795055, dtype: float64

In [211]:
small_Rank

Series([], Name: PE, dtype: float64)

In [213]:
middle

1898927.0

In [218]:
df_merge_Year[df_merge_Year['PE'] > 12.5]['PE']

0          13.3
1          13.2
2          13.2
3          13.7
4          13.5
           ... 
3797726    14.2
3797727    14.4
3797728    14.4
3797729    14.4
3797730    14.6
Name: PE, Length: 2344422, dtype: float64

# main strategy : PE(因子值排序加權) tri

In [231]:
import warnings
warnings.filterwarnings("ignore")

In [261]:
#basic settings
weight_method_list = Factor_args['weight_method']
factor_list = Factor_args[Main_Factor_Name]['factors']
longbig_list = Factor_args[Main_Factor_Name]['longbig']
Date_list = np.sort(df_merge_Year['Date'].unique())

factor  = factor_list[0]
longbig = longbig_list[0]
weight_method   = 'tri'
initial_account = 1000000000

In [275]:
df_Final = pd.DataFrame()
need_col = ['_Nstock_', '_Amount_', '_Weight_']
need_col = [factor + x + weight_method for x in need_col]
need_col_less1 = [x + '_t-1' for x in need_col]
#progress = ProgressBar()
for i in range(len(Date_list)):
    #day 1
    if i == 0:
        #2007/01/04
        Date = Date_list[i]
        #找出所有day1的data
        df_MV = df_merge_Year[df_merge_Year.Date == Date]
        #篩選前200大個股
        df_MV = df_MV[df_MV.MV_Select == 1]
        #將t-1期的PE由大到小排
        df_MV = df_MV.sort_values(factor + '_t-1', ascending = False)
        #給予前200大個股排名 1~200
        df_MV[factor + '_Rank'] = np.arange(len(df_MV)) + 1
        #factor -> PE(賣因子大的，買因子小的:longbid = 0)、no long or short index、using因子值排序加權
        df_MV[factor + '_Weight_' + weight_method] = cal_3group_weight_tri(df_MV, factor, longbig, 
                                                                           short_index=False, long_index=False, value=False) 
        df_MV = df_MV[['Code', 'Date', 'Close', 'Close_t+1', 
                       factor+'_t-1', factor+'_t-2', factor+'_Rank', factor+'_Weight_'+weight_method]]
        #numbers of stock、amount
        df_MV[factor + '_Nstock_' + weight_method], df_MV[factor + '_Amount_' + weight_method] = Cal_Amount(df_MV, factor, weight_method)
        df_Final = df_MV
    #day 1 之後
    else:
        #df_less1 -> 昨天的df_MV
        df_less1 = df_MV
        Date = Date_list[i]
        df_MV = df_merge_Year[df_merge_Year.Date == Date]
        df_MV = df_MV[df_MV.MV_Select == 1]
        df_MV = df_MV.sort_values(factor + '_t-1', ascending=False)
        df_MV[factor + '_Rank'] = np.arange(len(df_MV)) + 1
        #factor -> PE(賣因子大的，買因子小的:longbid = 0)、no long or short index、using因子值排序加權
        df_MV[factor + '_Weight_' + weight_method] = cal_3group_weight_tri(df_MV, factor, longbig, 
                                                                           short_index=False, long_index=False, value=False) 
        df_MV = df_MV[['Code', 'Date', 'Close', 'Close_t+1', 
                       factor+'_t-1', factor+'_t-2', factor+'_Rank', factor+'_Weight_'+weight_method]]
        #numbers of stock、amount
        df_MV[factor+'_Nstock_'+weight_method], df_MV[factor+'_Amount_'+weight_method]  = Cal_Amount(df_MV, factor, weight_method)
        #today -> t-1
        df_less1[need_col_less1 + [factor + '_Rank_t-1', 'Close_t+1_less1']] = df_less1[need_col + [factor + '_Rank','Close_t+1']]
        df_MV_m = pd.merge(left = df_MV,right = df_less1[need_col_less1+['Code',factor+'_Rank_t-1','Close_t+1_less1']], 
                           on=['Code'], how='outer')
        #fillna -> date由前往後補
        df_MV_m['Date'] = df_MV_m['Date'].fillna(method = 'ffill')
        #fillna -> close price將值補為close_t+1_less1
        df_MV_m['Close']= df_MV_m['Close'].fillna(df_MV_m['Close_t+1_less1'])
        #剩餘的將na轉為0
        df_MV_m = df_MV_m.fillna(0)
        df_Final = pd.concat([df_Final, df_MV_m], axis = 0)

In [321]:
df_Final

,Code,Date,Close,Close_t+1,PE_t-1,PE_t-2,PE_Rank,PE_Weight_tri,PE_Nstock_tri,PE_Amount_tri,PE_Nstock_tri_t-1,PE_Amount_tri_t-1,PE_Weight_tri_t-1,PE_Rank_t-1,Close_t+1_less1
856805,2362,20070104.0,30.50,29.45,647.9,647.9,1.0,-0.019802,-324.0,-9882000.0,-324.0,-9882000.0,-0.019802,1.0,29.45
706511,2107,20070104.0,23.50,23.10,92.0,94.4,2.0,-0.019604,-417.0,-9799500.0,-417.0,-9799500.0,-0.019604,2.0,23.10
1240222,2609,20070104.0,18.00,17.75,87.8,87.8,3.0,-0.019406,-539.0,-9702000.0,-539.0,-9702000.0,-0.019406,3.0,17.75
829945,2352,20070104.0,17.85,17.85,59.8,59.3,4.0,-0.019208,-538.0,-9603300.0,-538.0,-9603300.0,-0.019208,4.0,17.85
1241662,2610,20070104.0,15.20,15.05,56.1,56.7,5.0,-0.019010,-625.0,-9500000.0,-625.0,-9500000.0,-0.019010,5.0,15.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,2014,20190927.0,9.74,0.00,8.6,8.7,188.0,0.019759,1015.0,9886100.0,1004.0,9889400.0,0.019759,188.0,9.74
188,2888,20190927.0,9.39,0.00,7.8,7.7,189.0,0.019974,1064.0,9990960.0,1045.0,9990200.0,0.019974,189.0,9.39
189,2534,20190927.0,19.90,0.00,7.7,7.7,190.0,0.020189,508.0,10109200.0,505.0,10100000.0,0.020189,190.0,19.90
190,5534,20190927.0,79.50,0.00,7.7,7.7,191.0,0.020404,129.0,10255500.0,129.0,10255500.0,0.020404,191.0,79.50


In [291]:
initial_account/2*df_Final['PE_Weight_tri']/df_Final['Close']/1000

856805     -324.622626
706511     -417.105540
1240222    -539.053905
829945     -538.036997
1241662    -625.325690
              ...     
187        1014.345491
188        1063.590352
189         507.261393
190         128.325661
191         569.573390
Length: 614291, dtype: float64

In [296]:
#交易紀錄存成txt檔
df_Final.to_csv('./Weight/'+factor+'_Weight_'+weight_method+'_new_'+'.txt',index=False)

# performance

In [298]:
df_performance = df_Final.reset_index(drop = True)
df_Equity_Trunover_Tax = pd.DataFrame()

In [300]:
#計算出每一天的profit，最後要算port value時，再以initial account+逐步累加的profit
df_Equity_Trunover_Tax[['Date','Profit_'+weight_method]] = df_performance.groupby('Date').apply(lambda x : Cal_Profit(x,factor,weight_method)).reset_index()
df_Equity_Trunover_Tax

,Date,Profit_tri
0,20070104.0,3510020.0
1,20070105.0,4778550.0
2,20070108.0,2418120.0
3,20070109.0,-275190.0
4,20070110.0,-2684260.0
...,...,...
3141,20190923.0,471790.0
3142,20190924.0,1811930.0
3143,20190925.0,1057580.0
3144,20190926.0,742150.0


In [301]:
#TAX
df_Equity_Trunover_Tax['Tax_'+weight_method] = df_performance.groupby('Date').apply(lambda x : 
                                                                        Cal_Cost(x,factor ,weight_method)).reset_index(drop=True)
df_Equity_Trunover_Tax

,Date,Profit_tri,Tax_tri
0,20070104.0,3510020.0,0.00
1,20070105.0,4778550.0,107635.65
2,20070108.0,2418120.0,81257.97
3,20070109.0,-275190.0,88423.77
4,20070110.0,-2684260.0,80839.89
...,...,...,...
3141,20190923.0,471790.0,52424.79
3142,20190924.0,1811930.0,50618.79
3143,20190925.0,1057580.0,56465.28
3144,20190926.0,742150.0,65421.30


In [302]:
df_Equity_Trunover_Tax['Turnover_'+weight_method] = df_performance.groupby('Date').apply(lambda x : 
                                                                        Cal_Turnover(x,factor ,weight_method)).reset_index(drop=True)
df_Equity_Trunover_Tax

,Date,Profit_tri,Tax_tri,Turnover_tri
0,20070104.0,3510020.0,0.00,0.0
1,20070105.0,4778550.0,107635.65,68037590.0
2,20070108.0,2418120.0,81257.97,49384890.0
3,20070109.0,-275190.0,88423.77,56837180.0
4,20070110.0,-2684260.0,80839.89,53086370.0
...,...,...,...,...
3141,20190923.0,471790.0,52424.79,34065570.0
3142,20190924.0,1811930.0,50618.79,35061900.0
3143,20190925.0,1057580.0,56465.28,34299270.0
3144,20190926.0,742150.0,65421.30,42563580.0


In [303]:
df_Equity_Trunover_Tax['Equity_'+weight_method] = initial_account + df_Equity_Trunover_Tax['Profit_'+weight_method].cumsum() - df_Equity_Trunover_Tax['Tax_'+weight_method].cumsum()
df_Equity_Trunover_Tax

,Date,Profit_tri,Tax_tri,Turnover_tri,Equity_tri
0,20070104.0,3510020.0,0.00,0.0,1.003510e+09
1,20070105.0,4778550.0,107635.65,68037590.0,1.008181e+09
2,20070108.0,2418120.0,81257.97,49384890.0,1.010518e+09
3,20070109.0,-275190.0,88423.77,56837180.0,1.010154e+09
4,20070110.0,-2684260.0,80839.89,53086370.0,1.007389e+09
...,...,...,...,...,...
3141,20190923.0,471790.0,52424.79,34065570.0,1.829913e+09
3142,20190924.0,1811930.0,50618.79,35061900.0,1.831674e+09
3143,20190925.0,1057580.0,56465.28,34299270.0,1.832675e+09
3144,20190926.0,742150.0,65421.30,42563580.0,1.833352e+09


In [352]:
#setting risk free rate
df_RF = pd.read_excel('Data/3.Rm_Rf/2009-201909Rf.xlsx')
df_RF.columns = df_RF.iloc[3, ]
df_RF = df_RF.drop([0, 1, 2, 3])
df_RF = df_RF.dropna(axis = 1).reset_index(drop = True)
df_RF.columns = ['Date', 'Rf']
df_RF['Date'] = [datetime.strftime(x,format='%Y%m%d') for x in df_RF['Date']]
#drop duplicate -> 刪除重複的日期
df_RF = df_RF.sort_values('Date').drop_duplicates()
#rf 取平均值
E_RF = df_RF['Rf'].mean()/100

In [384]:
#performance
def performance_stats(equity, E_RF):
    equity = pd.DataFrame(equity)
    #年化收益
    CAGR  = ((equity.iloc[-1]/equity.iloc[0]-1)*(250/len(equity)))[0]
    #年化標準差
    STD_Y = (equity.pct_change().std()*np.sqrt(250))[0]
    #夏普值
    Sharp_ratio = (CAGR - E_RF)/STD_Y
    #最大回撤MDD 回撤率mdd
    equity = pd.DataFrame(equity)
    Cum_R = equity.values/(equity.head(1).values)
    Cum_R = pd.DataFrame(Cum_R)
    #cummax -> 累積最大值
    D = Cum_R.cummax() - Cum_R
    #MDD = 累積差距最大的 * 10億 -> 虧最多的錢
    MDD = D.max()*initial_account
    d = D / (D +Cum_R)
    mdd = d.max()
    #NP/MDD -> 最後賺到多少 / 虧最多為多少
    NP = pd.DataFrame(equity.iloc[-1] - equity.iloc[0])
    NP_MDD = NP / MDD
    #勝率
    R = equity.pct_change()
    Odd = R[R > 0].count() / len(equity)
    #偏態
    skew = equity.pct_change().skew()
    return CAGR, STD_Y, Sharp_ratio, mdd, NP_MDD, Odd, skew

In [385]:
CAGR, STD_Y, Sharp_ratio, mdd, NP_MDD, Odd, skew = performance_stats(df_Equity_Trunover_Tax['Equity_tri'], E_RF)

In [390]:
skew

Equity_tri    0.215552
dtype: float64

# main strategy : 10組固定 tri_10g

In [391]:
import warnings
warnings.filterwarnings("ignore")

In [398]:
weight_method_list = Factor_args['weight_method']
factor_list = Factor_args[Main_Factor_Name]['factors']
longbig_list = Factor_args[Main_Factor_Name]['longbig']
Date_list = np.sort(df_merge_Year['Date'].unique())
#setting
factor = factor_list[0]
longbig = longbig_list[0]
weight_method = 'tri_10g'
initial_account = 1000000000

df_Final = pd.DataFrame()
need_col = ['_Nstock_', '_Amount_', '_Weight_']
need_col = [factor + x + weight_method for x in need_col]
need_col_less1 = [x + '_t-1' for x in need_col]

In [399]:
need_col 

['PE_Nstock_tri_10g', 'PE_Amount_tri_10g', 'PE_Weight_tri_10g']

In [400]:
need_col_less1

['PE_Nstock_tri_10g_t-1', 'PE_Amount_tri_10g_t-1', 'PE_Weight_tri_10g_t-1']